# Setting

In [15]:
import os
import glob
import math
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.auto import tqdm
import random as r
import wandb
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Seed the behavior of the environment variable
os.environ['PYTHONHASHSEED'] = str(1)
# Seed numpy's instance in case you are using numpy's random number generator, shuffling operations, ...
np.random.seed(1)
# Seed Python's random number generator, in case you are using Python's random number generator, shuffling operations, ...
r.seed(1)



In [ ]:
wandb.init(project='mask_classification', entity='sala0320', reinit=True)
config = wandb.config

In [4]:
BASE_MASK_PATH = '/opt/ml/input/data/'
TRAIN_MASK_IMAGE_PATH = os.path.join(BASE_MASK_PATH, 'train/images/')
TRAIN_MASK_LABEL_PATH = os.path.join(BASE_MASK_PATH, 'train/train.csv')
TEST_MASK_IMAGE_PATH = os.path.join(BASE_MASK_PATH, 'test/images/')
TEST_MASK_LABEL_PATH = os.path.join(BASE_MASK_PATH, 'test/info.csv')

In [5]:
train_data = pd.read_csv('/opt/ml/input/data/train/train.csv')
train_pre_data = pd.read_csv('/opt/ml/input/data/train/train_pre.csv')
train_pre_data['gender'] = train_pre_data['gender'].map({'male':0, 'female':1})
train_pre_data.head()

,id,gender,race,age,path,class
0,000001,1,Asian,45,000001_female_Asian_45/normal.jpg,16
1,000001,1,Asian,45,000001_female_Asian_45/incorrect_mask.jpg,10
2,000001,1,Asian,45,000001_female_Asian_45/mask1.jpg,4
3,000001,1,Asian,45,000001_female_Asian_45/mask2.jpg,4
4,000001,1,Asian,45,000001_female_Asian_45/mask3.jpg,4


# Data

## Dataset

In [6]:
img_path = train_pre_data['path']
labels = train_pre_data['class']

In [7]:
class TrainDataset(Dataset):
    def __init__(self, img_paths, labels, transform):
        self.img_paths = TRAIN_MASK_IMAGE_PATH + img_paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        x = Image.open(self.img_paths[idx])
        y = self.labels[idx]
  
        if self.transform:
            x = self.transform(x)
        
        return x, y
        
    def __len__(self):
        return len(self.img_paths)

In [8]:
# train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(img_path, labels, test_size=0.2, stratify=labels, shuffle=True, random_state=30)
x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
x_valid = x_valid.reset_index(drop=True)
y_valid = y_valid.reset_index(drop=True)

In [11]:
#transform
train_transform = transforms.Compose([ 
    # transforms.Resize((380,380)),
    transforms.CenterCrop(380),
    # transforms.GaussianBlur(11),
    # transforms.RandomAutocontrast(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

valid_transform = transforms.Compose([
    transforms.CenterCrop(380),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
mask_train_dataset = TrainDataset(x_train, y_train, train_transform)
mask_valid_dataset = TrainDataset(x_valid, y_valid, valid_transform)

In [ ]:
from albumentations import *
from albumentations.pytorch import ToTensorV2


## DataLoader

In [12]:
from multiprocessing import cpu_count
num_workers = int(cpu_count() / 2)
batch_size = 32

mask_train_dataloader = DataLoader(
    mask_train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
    )

mask_valid_dataloader = DataLoader(
    mask_valid_dataset,
    batch_size=batch_size,
    )


In [13]:
images, labels = next(iter(mask_train_dataloader))
labels

tensor([ 4,  4, 14,  4, 16,  4,  3,  3,  3,  4,  4,  6,  4,  0,  4, 12,  4,  0,
         0,  4,  4,  1,  3,  3,  0,  3,  3,  4,  3, 16,  0,  0])

# Modeling

In [13]:
efficient = EfficientNet.from_pretrained('efficientnet-b4')
for param in efficient.parameters():
    param.requires_grad = False
efficient._fc = torch.nn.Linear(in_features=1792, out_features=18, bias=True)

# torch.nn.init.xavier_uniform_(efficient._fc.weight)
# stdv = 1. / math.sqrt(efficient._fc.weight.size(1))
# efficient._fc.bias.data.uniform_(-stdv, stdv)

model = efficient
model

Loaded pretrained weights for efficientnet-b4


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

# Train

In [ ]:
class FocalLoss(nn.Module):

    def __init__(self, gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        #print(self.gamma)
        self.eps = eps
        self.ce = torch.nn.CrossEntropyLoss(reduction="none")

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()


In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

model.to(device)

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 20 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션. 여러 값을 사용해보세요1

loss_fn = torch.nn.CrossEntropyLoss()
# loss_fn = FocalLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer,
#                                 lr_lambda=lambda epoch: 0.95 ** epoch,
#                                 last_epoch=-1,
#                                 verbose=False)
# config = wandb.config
# config.learning_rate = LEARNING_RATE

dataloaders = {
    "train" : mask_train_dataloader,
    "valid" : mask_valid_dataloader
}

cuda:0 is using!


In [ ]:
### 학습 코드 시작
best_test_accuracy = 0.
best_test_loss = 0.
# early_stopping = 0.

for epoch in range(NUM_EPOCH):
  for phase in ["train", "valid"]:
    running_loss = 0.
    running_acc = 0.
    running_f1 = 0.

    if phase == "train":
      model.train() 
      # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "valid":
      model.eval() 
      # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함
  
    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):

      images = images.to(device)
      labels = labels.to(device)
      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트
          # scheduler.step() 

      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장
      running_f1 += f1_score(preds.cpu().numpy(), labels.cpu().numpy(), average='macro')
      
    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)
    wandb.log({"epoch_loss":  epoch_loss.item()})
    wandb.log({"epoch_acc":  epoch_acc.item()})

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")

    if phase == "valid" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc

    if phase == "valid" and best_test_loss < epoch_loss: # phase가 test일 때, best loss 계산
      best_test_loss = epoch_loss
      torch.save(model.state_dict(), '/opt/ml/input/data/train/best_model.pth')
      
    # else:
    #   early_stopping += 1
    #   print("Early stopping! " + str(early_stopping))
    #   if early_stopping >=5:
    #     break

    
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")



# Testing

In [9]:
model = EfficientNet.from_pretrained('efficientnet-b4')
model._fc = torch.nn.Linear(in_features=1792, out_features=18, bias=True)
model.load_state_dict(torch.load('/opt/ml/input/data/train/best_model.pth'))
model

Loaded pretrained weights for efficientnet-b4


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [20]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = TRAIN_MASK_IMAGE_PATH + img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [21]:
# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = train_pre_data['path']
transform = transforms.Compose([
    transforms.CenterCrop(380),
    # Resize((380, 380), Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
# model = MyModel(num_classes=18).to(device)
model = model.to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())

In [29]:
test_pre_data = train_pre_data
test_pre_data['predict'] = all_predictions
for i in range(len(test_pre_data)):
    if test_pre_data['class'][i] != test_pre_data['predict'][i]:
        print(f"path : {test_pre_data['path'][i]} real : {test_pre_data['class'][i]} predict : {test_pre_data['predict'][i]}")

path : 000001_female_Asian_45/incorrect_mask.jpg real : 10 predict : 4
path : 000001_female_Asian_45/mask1.jpg real : 4 predict : 3
path : 000001_female_Asian_45/mask2.jpg real : 4 predict : 3
path : 000002_female_Asian_52/normal.jpg real : 16 predict : 4
path : 000002_female_Asian_52/incorrect_mask.jpg real : 10 predict : 4
path : 000004_male_Asian_54/incorrect_mask.jpg real : 7 predict : 4
path : 000004_male_Asian_54/mask1.jpg real : 1 predict : 4
path : 000004_male_Asian_54/mask2.jpg real : 1 predict : 4
path : 000004_male_Asian_54/mask3.jpg real : 1 predict : 4
path : 000004_male_Asian_54/mask4.jpg real : 1 predict : 4
path : 000004_male_Asian_54/mask5.jpg real : 1 predict : 4
path : 000005_female_Asian_58/incorrect_mask.jpg real : 10 predict : 4
path : 000006_female_Asian_59/incorrect_mask.jpg real : 10 predict : 4
path : 000007_female_Asian_58/incorrect_mask.jpg real : 10 predict : 4
path : 000008_female_Asian_58/incorrect_mask.jpg real : 10 predict : 4
path : 000009_female_Asian

In [ ]:
import matplotlib.pyplot as plt
imgs = submission['ImageID']
label = submission['ans']
fig, axes = plt.subplots(1, 5, sharex=True, sharey=True, figsize=(12, 6))
for i, j in enumerate(range(5)):
    axes[i].imshow(Image.open('/opt/ml/input/data/eval/images/' + imgs[j]))
    axes[i].set_title(f'{label[j]}')
plt.show()